In [4]:
from dotenv import load_dotenv

load_dotenv("../.env.prod")
import os
from utils.chunker import chunker, chunk_single_text
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import json
import numpy as np


In [5]:
ds = pd.read_parquet("../files/processed/final_datasets/val.parquet")
ds.columns

Index(['candidate_id', 'vacant_id', 't_apply', 'stage_max', 'publish_date',
       'label', 'vacant_full_text', 'vacant_city_ids', 'candidate_full_text',
       'candidate_city_id', 'candidate_fourier_features',
       'no_valid_vacant_city_ids', 'selected_city_id', 'selected_distance',
       'exact_match', 'vacant_fourier_feature', 'job_chunks_input_ids',
       'job_chunks_attention_mask', 'cand_chunks_input_ids',
       'cand_chunks_attention_mask'],
      dtype='object')

In [6]:
import numpy as np
import pandas as pd

# start from ds
df = ds.copy()

# keep only 0/1 labels just in case
df = df[df['label'].isin([0, 1])].copy()

pairs = []

for vac_id, g in df.groupby('vacant_id'):
    pos = g[g['label'] == 1].sort_values('t_apply')
    neg = g[g['label'] == 0].sort_values('t_apply')
    
    # skip vacancies with no pos or no neg
    if pos.empty or neg.empty:
        continue
    
    pos = pos.reset_index(drop=True)
    neg = neg.reset_index(drop=True)
    
    n_pos = len(pos)
    n_neg = len(neg)
    

    neg_idx = np.tile(np.arange(n_neg), int(np.ceil(n_pos / n_neg)))[:n_pos]
    neg_rep = neg.iloc[neg_idx].reset_index(drop=True)
    
    pair = pd.DataFrame({
        'vacant_id': vac_id,
        'pos_candidate_id': pos['candidate_id'].values,
        'neg_candidate_id': neg_rep['candidate_id'].values,
    })
    

    other_cols = [c for c in df.columns if c not in ['candidate_id', 'vacant_id']]
    
    for col in other_cols:
        pair['pos_' + col] = pos[col].values
        pair['neg_' + col] = neg_rep[col].values
    
    pairs.append(pair)

pairs_df = pd.concat(pairs, ignore_index=True)


In [7]:
pairs_df.to_parquet("../files/processed/paired_datasets/val.parquet")

In [8]:
from dotenv import load_dotenv

load_dotenv("../.env.prod")
import os
from utils.chunker import chunker, chunk_single_text
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import json
import numpy as np

ds = pd.read_parquet("../files/processed/final_datasets/test.parquet")
ds.columns
import numpy as np
import pandas as pd

# start from ds
df = ds.copy()

# keep only 0/1 labels just in case
df = df[df['label'].isin([0, 1])].copy()

pairs = []

for vac_id, g in df.groupby('vacant_id'):
    pos = g[g['label'] == 1].sort_values('t_apply')
    neg = g[g['label'] == 0].sort_values('t_apply')
    
    # skip vacancies with no pos or no neg
    if pos.empty or neg.empty:
        continue
    
    pos = pos.reset_index(drop=True)
    neg = neg.reset_index(drop=True)
    
    n_pos = len(pos)
    n_neg = len(neg)
    

    neg_idx = np.tile(np.arange(n_neg), int(np.ceil(n_pos / n_neg)))[:n_pos]
    neg_rep = neg.iloc[neg_idx].reset_index(drop=True)
    
    pair = pd.DataFrame({
        'vacant_id': vac_id,
        'pos_candidate_id': pos['candidate_id'].values,
        'neg_candidate_id': neg_rep['candidate_id'].values,
    })
    

    other_cols = [c for c in df.columns if c not in ['candidate_id', 'vacant_id']]
    
    for col in other_cols:
        pair['pos_' + col] = pos[col].values
        pair['neg_' + col] = neg_rep[col].values
    
    pairs.append(pair)

pairs_df = pd.concat(pairs, ignore_index=True)

pairs_df.to_parquet("../files/processed/paired_datasets/test.parquet")

In [ ]:
df

,candidate_id,vacant_id,t_apply,stage_max,publish_date,label,scenario,vacant_city_loc,vacant_full_text,vacant_city_ids,vacant_remote,candidate_full_text,candidate_city_loc,candidate_city_id,candidate_fourier_features,no_valid_vacant_city_ids,selected_city_id,selected_distance,exact_match,vacant_fourier_feature
0,319,502825,2025-06-24 19:54:11.760000+00:00,1.0,1746204569247,0,cold_v,"[(4.316107698,-74.181072702)]",Auxiliar Administrativo de Obra - Sector Const...,[17050],False,Administradora Financiera y de Sistemas. Poseo...,"(4.316107698,-74.181072702)",17050,"[0.2214912176, -0.46965277190000004, 0.3965891...",False,17050,0.000000,True,"[0.2214912176, -0.46965277190000004, 0.3965891..."
1,319,504093,2025-06-17 15:08:55.866000+00:00,1.0,1746467371271,0,cold_v,"[(4.316107698,-74.181072702)]",Asistente Administrativo con Enfoque Operativo...,[17050],False,Administradora Financiera y de Sistemas. Poseo...,"(4.316107698,-74.181072702)",17050,"[0.2214912176, -0.46965277190000004, 0.3965891...",False,17050,0.000000,True,"[0.2214912176, -0.46965277190000004, 0.3965891..."
2,319,506062,2025-09-30 16:26:07.138000+00:00,1.0,1746647241149,0,cold_v,"[(4.316107698,-74.181072702)]",¡Bienvenido a nuestra oferta en KPMG! Te cuent...,[17050],False,Administradora Financiera y de Sistemas. Poseo...,"(4.316107698,-74.181072702)",17050,"[0.2214912176, -0.46965277190000004, 0.3965891...",False,17050,0.000000,True,"[0.2214912176, -0.46965277190000004, 0.3965891..."
3,319,509988,2025-10-07 12:27:42.682000+00:00,1.0,1747765739580,0,cold_v,"[(6.358841086,-75.589735878), (6.351882893,-75...",¡Únete a nuestro equipo comercial en Seguros S...,"[16911, 16934, 16940, 16952, 16957, 16963, 169...",False,Administradora Financiera y de Sistemas. Poseo...,"(4.316107698,-74.181072702)",17050,"[0.2214912176, -0.46965277190000004, 0.3965891...",False,17050,0.000000,True,"[0.2214912176, -0.46965277190000004, 0.3965891..."
4,319,511464,2025-06-23 22:34:31.797000+00:00,1.0,1747328590510,0,cold_v,"[(4.316107698,-74.181072702)]",Auxiliar Administrativo Comercial Palabras cla...,[17050],False,Administradora Financiera y de Sistemas. Poseo...,"(4.316107698,-74.181072702)",17050,"[0.2214912176, -0.46965277190000004, 0.3965891...",False,17050,0.000000,True,"[0.2214912176, -0.46965277190000004, 0.3965891..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42729,8850776,505026,2025-07-10 05:03:45.486000+00:00,1.0,1746543119326,0,zero,"[(10.463433617,-75.45889915)]",Asistente de Operaciones ¡Tu gateway hacia la...,[17059],False,"Soy una persona organizada, responsable y con ...","(10.463433617,-75.45889915)",17059,"[0.0592304878, -0.4938886166, 0.0988287479, 0....",False,17059,0.000000,True,"[0.0592304878, -0.4938886166, 0.0988287479, 0...."
42730,8850776,512628,2025-07-10 05:03:47.859000+00:00,1.0,1747435983454,0,zero,"[(10.463433617,-75.45889915)]",Auxiliar Administrativo en Cartagena Palabras ...,[17059],False,"Soy una persona organizada, responsable y con ...","(10.463433617,-75.45889915)",17059,"[0.0592304878, -0.4938886166, 0.0988287479, 0....",False,17059,0.000000,True,"[0.0592304878, -0.4938886166, 0.0988287479, 0...."
42731,8850836,509332,2025-07-16 18:32:39.021000+00:00,1.0,1747177244526,0,zero,"[(0.497708465,-77.240476252)]",Administrador\nde Punto de Venta ¿Qué harás? A...,[17648],False,Soy una persona con muchas capacidades y habil...,"(4.316107698,-74.181072702)",17050,"[0.2214912176, -0.46965277190000004, 0.3965891...",False,17648,0.735910,False,"[0.3253635466, -0.38770428300000004, -0.263783..."
42732,8850852,500757,2025-07-02 14:04:16.843000+00:00,1.0,1745935719401,0,zero,"[(7.270560088,-72.652121526)]",Líder de Tienda Farmatodo Cúcuta Palabras clav...,[49992],False,Responsable. experiencia: Manejo de mercancía....,"(7.270560088,-72.652121526)",49992,"[0.13676705960000002, -0.4962901771, 0.4686130...",False,49992,0.000000,True,"[0.13676705960000002, -0.4962901771, 0.4686130..."


In [ ]:
pairs_df[["vacant_id", 'pos_candidate_id', 'neg_candidate_id',  ]]

Index(['vacant_id', 'pos_candidate_id', 'neg_candidate_id', 'pos_t_apply',
       'neg_t_apply', 'pos_stage_max', 'neg_stage_max', 'pos_publish_date',
       'neg_publish_date', 'pos_label', 'neg_label', 'pos_scenario',
       'neg_scenario', 'pos_vacant_city_loc', 'neg_vacant_city_loc',
       'pos_vacant_full_text', 'neg_vacant_full_text', 'pos_vacant_city_ids',
       'neg_vacant_city_ids', 'pos_vacant_remote', 'neg_vacant_remote',
       'pos_candidate_full_text', 'neg_candidate_full_text',
       'pos_candidate_city_loc', 'neg_candidate_city_loc',
       'pos_candidate_city_id', 'neg_candidate_city_id',
       'pos_candidate_fourier_features', 'neg_candidate_fourier_features',
       'pos_no_valid_vacant_city_ids', 'neg_no_valid_vacant_city_ids',
       'pos_selected_city_id', 'neg_selected_city_id', 'pos_selected_distance',
       'neg_selected_distance', 'pos_exact_match', 'neg_exact_match',
       'pos_vacant_fourier_feature', 'neg_vacant_fourier_feature'],
      dtype='object'